# Coursework Template

**Dependencies and imports**

This can take a minute...

In [ ]:
# !pip install swig
# !pip install --upgrade rldurham

In [ ]:
import torch
import rldurham as rld

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from collections import deque

**Reinforcement learning agent**

Replace this with your own agent, I recommend starting with TD3 (lecture 8).

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Normal
from collections import deque

class ActorCritic(nn.Module):
    def __init__(self, num_inputs, num_outputs, hidden_size=400):
        """
        Initialize the Actor-Critic network with separate policy and value heads.
        Uses larger networks and layer normalization for better stability.
        
        Args:
            num_inputs (int): Dimension of state space
            num_outputs (int): Dimension of action space
            hidden_size (int): Number of neurons in hidden layers
        """
        super(ActorCritic, self).__init__()
        
        # Shared feature extractor with layer normalization for better training stability
        self.feature_extractor = nn.Sequential(
            nn.Linear(num_inputs, hidden_size),
            nn.LayerNorm(hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, hidden_size),
            nn.LayerNorm(hidden_size),
            nn.Tanh()
        )
        
        # Separate networks for policy and value to prevent interference
        self.policy_network = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.LayerNorm(hidden_size // 2),
            nn.Tanh()
        )
        
        self.value_network = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.LayerNorm(hidden_size // 2),
            nn.Tanh()
        )
        
        # Policy head with smaller initialization for better initial exploration
        self.policy_mean = nn.Linear(hidden_size // 2, num_outputs)
        torch.nn.init.uniform_(self.policy_mean.weight, -1e-3, 1e-3)
        torch.nn.init.uniform_(self.policy_mean.bias, -1e-3, 1e-3)
        
        # Value head
        self.value = nn.Linear(hidden_size // 2, 1)

    def forward(self, state):
        """
        Forward pass through the network with separate paths for policy and value
        
        Args:
            state (torch.Tensor): Current state
            
        Returns:
            tuple: (action_mean, state_value)
        """
        features = self.feature_extractor(state)
        policy_features = self.policy_network(features)
        value_features = self.value_network(features)
        
        action_mean = torch.tanh(self.policy_mean(policy_features))  # Constrain actions to [-1, 1]
        state_value = self.value(value_features)
        
        return action_mean, state_value

class PPOAgent:
    def __init__(self, num_inputs, num_outputs, hidden_size=400, lr=2e-4, 
                 gamma=0.99, epsilon=0.2, value_coef=0.5, entropy_coef=0.01):
        """
        Initialize the PPO agent with optimized hyperparameters for BipedalWalker
        
        Args:
            num_inputs (int): Dimension of state space
            num_outputs (int): Dimension of action space
            hidden_size (int): Number of neurons in hidden layers
            lr (float): Learning rate
            gamma (float): Discount factor
            epsilon (float): PPO clipping parameter
            value_coef (float): Value loss coefficient
            entropy_coef (float): Entropy bonus coefficient
        """
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.num_outputs = num_outputs
        
        # Initialize neural network and optimizer
        self.ac_net = ActorCritic(num_inputs, num_outputs, hidden_size).to(self.device)
        self.optimizer = optim.Adam(self.ac_net.parameters(), lr=lr)
        
        # Initialize hyperparameters
        self.gamma = gamma
        self.epsilon = epsilon
        self.value_coef = value_coef
        self.entropy_coef = entropy_coef
        
        # Initialize action standard deviation with more conservative initial value
        self.action_std = 0.5
        self.min_action_std = 0.05
        self.action_std_decay_rate = 0.03
        self.action_std_decay_freq = 200  # Decay frequency in episodes
        
        # Initialize memory with separate recent trajectory buffer
        self.data = []
        self.trajectory_length = 0
        self.max_trajectory_length = 2048  # Maximum steps before forcing an update
        
        # Statistics tracking with longer history
        self.rewards_history = deque(maxlen=200)
        self.episode_count = 0
        
        # Store the most recent action and log probability for training
        self.last_action = None
        self.last_log_prob = None
        
        # Advantage normalization tracking
        self.advantage_mean = 0
        self.advantage_std = 1
    
    def sample_action(self, state):
        """
        Sample an action from the policy with noise reduction over time
        
        Args:
            state (numpy.ndarray): Current state
            
        Returns:
            list: List of action values as Python floats
        """
        state = np.array(state, dtype=np.float32)
        state_tensor = torch.FloatTensor(state).unsqueeze(0).to(self.device)
        
        with torch.no_grad():
            action_mean, _ = self.ac_net(state_tensor)
            
            # Add noise for exploration
            std = torch.full_like(action_mean, self.action_std)
            dist = Normal(action_mean, std)
            action = dist.sample()
            
            # Calculate log probability for training
            log_prob = dist.log_prob(action).sum(dim=-1)
            
            # Convert to numpy and store for training
            action_numpy = action.cpu().numpy().squeeze()
            self.last_action = action_numpy
            self.last_log_prob = log_prob.cpu().numpy()
            
            # Convert to list of Python floats
            action_list = [float(a) for a in action_numpy]
            
            return action_list
    
    def put_data(self, state, action, reward, next_state, done):
        """
        Store transition in memory with reward scaling
        
        Args:
            state (numpy.ndarray): Current state
            action (list): Action taken (list of floats)
            reward (float): Reward received
            next_state (numpy.ndarray): Next state
            done (bool): Whether episode is done
        """
        # Scale reward for better training stability
        scaled_reward = reward * 0.1  # Scale down rewards to prevent value function instability
        
        # Convert action list back to numpy array for storage
        action_array = np.array(action, dtype=np.float32)
        self.data.append((state, action_array, self.last_log_prob, scaled_reward, next_state, done))
        
        self.trajectory_length += 1
    
    def train(self):
        """
        Train the agent using PPO with several optimizations
        
        Returns:
            tuple: (policy_loss, value_loss)
        """
        # Skip training if not enough data
        if len(self.data) < 32:  # Reduced minimum batch size for early learning
            return 0.0, 0.0
            
        # Adjust batch size based on available data
        batch_size = min(256, max(32, len(self.data) // 4))
            
        # Convert stored data to tensors
        states = torch.FloatTensor([t[0] for t in self.data]).to(self.device)
        actions = torch.FloatTensor([t[1] for t in self.data]).to(self.device)
        old_log_probs = torch.FloatTensor([t[2] for t in self.data]).to(self.device)
        rewards = torch.FloatTensor([t[3] for t in self.data]).to(self.device)
        next_states = torch.FloatTensor([t[4] for t in self.data]).to(self.device)
        dones = torch.FloatTensor([t[5] for t in self.data]).to(self.device)
        
        # Compute returns and advantages with GAE
        with torch.no_grad():
            _, next_values = self.ac_net(next_states)
            _, current_values = self.ac_net(states)
            next_values = next_values.squeeze()
            current_values = current_values.squeeze()
            
            # Calculate advantages using GAE
            advantages = torch.zeros_like(rewards)
            gae = 0
            for t in reversed(range(len(rewards))):
                if t == len(rewards) - 1:
                    next_value = next_values[t]
                else:
                    next_value = current_values[t + 1]
                    
                delta = rewards[t] + self.gamma * next_value * (1 - dones[t]) - current_values[t]
                gae = delta + self.gamma * 0.95 * (1 - dones[t]) * gae
                advantages[t] = gae
                
            returns = advantages + current_values
            
            # Normalize advantages
            advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)
        
        # PPO update with mini-batches
        batch_size = min(256, len(self.data))
        num_updates = 8  # Number of optimization epochs
        
        total_policy_loss = 0
        total_value_loss = 0
        
        for _ in range(num_updates):
            # Random sampling for mini-batches
            indices = np.random.permutation(len(self.data))
            
            for start in range(0, len(self.data), batch_size):
                end = start + batch_size
                batch_indices = indices[start:end]
                
                # Get batch data
                batch_states = states[batch_indices]
                batch_actions = actions[batch_indices]
                batch_old_log_probs = old_log_probs[batch_indices]
                batch_advantages = advantages[batch_indices]
                batch_returns = returns[batch_indices]
                
                # Get updated action probabilities and values
                action_means, values = self.ac_net(batch_states)
                dist = Normal(action_means, torch.full_like(action_means, self.action_std))
                new_log_probs = dist.log_prob(batch_actions).sum(dim=-1)
                
                # Compute ratio and clipped ratio
                ratio = torch.exp(new_log_probs - batch_old_log_probs)
                clipped_ratio = torch.clamp(ratio, 1-self.epsilon, 1+self.epsilon)
                
                # Compute losses with additional penalty for large actions
                policy_loss = -torch.min(ratio * batch_advantages, 
                                       clipped_ratio * batch_advantages).mean()
                value_loss = ((values.squeeze() - batch_returns) ** 2).mean()
                entropy_loss = -dist.entropy().mean()
                
                # Additional penalty for extreme actions
                action_penalty = 0.1 * (torch.max(torch.abs(action_means)) - 0.9).clamp(min=0)
                
                # Combined loss
                total_loss = (policy_loss + 
                             self.value_coef * value_loss + 
                             self.entropy_coef * entropy_loss +
                             action_penalty)
                
                # Optimize
                self.optimizer.zero_grad()
                total_loss.backward()
                torch.nn.utils.clip_grad_norm_(self.ac_net.parameters(), 0.5)
                self.optimizer.step()
                
                total_policy_loss += policy_loss.item()
                total_value_loss += value_loss.item()
        
        # Clear memory
        self.data = []
        self.trajectory_length = 0
        
        # Calculate number of actual updates performed
        num_batches = max(1, len(self.data) // batch_size)
        total_updates = num_updates * num_batches
        
        # Return average losses, protecting against division by zero
        if total_updates > 0:
            avg_policy_loss = total_policy_loss / total_updates
            avg_value_loss = total_value_loss / total_updates
        else:
            avg_policy_loss = total_policy_loss
            avg_value_loss = total_value_loss
        
        return avg_policy_loss, avg_value_loss
    
    def reset(self):
        """Reset episode-specific parameters"""
        self.last_action = None
        self.last_log_prob = None
        
        # Force training if trajectory is too long
        if self.trajectory_length >= self.max_trajectory_length:
            self.train()
            self.trajectory_length = 0
    
    def update_stats(self, episode_reward):
        """
        Update training statistics and adjust exploration
        
        Args:
            episode_reward (float): Total reward for the episode
        """
        self.rewards_history.append(episode_reward)
        self.episode_count += 1
        
        # More conservative action std decay
        if self.episode_count % self.action_std_decay_freq == 0:
            # Only decay if we're seeing improvement
            if np.mean(list(self.rewards_history)[-20:]) > -100:
                self.action_std = max(self.min_action_std, 
                                    self.action_std - self.action_std_decay_rate)
    
    def get_average_reward(self):
        """
        Get average reward over last 100 episodes
        
        Returns:
            float: Average reward
        """
        return np.mean(self.rewards_history)

**Prepare the environment and wrap it to capture statistics, logs, and videos**

In [ ]:
env = rld.make("rldurham/Walker", render_mode="rgb_array")
# env = rld.make("rldurham/Walker", render_mode="rgb_array", hardcore=True) # only attempt this when your agent has solved the non-hardcore version

# get statistics, logs, and videos
env = rld.Recorder(
    env,
    smoothing=10,                       # track rolling averages (useful for plotting)
    video=True,                         # enable recording videos
    video_folder="videos",              # folder for videos
    video_prefix="xxxx00-agent-video",  # prefix for videos (replace xxxx00 with your username)
    logs=True,                          # keep logs
)

# training on CPU recommended
rld.check_device()

# environment info
discrete_act, discrete_obs, act_dim, obs_dim = rld.env_info(env, print_out=True)

# render start image
env.reset(seed=42)
rld.render(env)

In [ ]:
# in the submission please use seed_everything with seed 42 for verification
seed, observation, info = rld.seed_everything(42, env)

# initialise agent
agent = PPOAgent(env.observation_space.shape[0], env.action_space.shape[0])
max_episodes = 1000
max_timesteps = 2000

# track statistics for plotting
tracker = rld.InfoTracker()

# switch video recording off (only switch on every x episodes as this is slow)
env.video = False

for episode in range(max_episodes):
    # Set up episode
    env.info = episode % 10 == 0
    env.video = episode % 10 == 0
    current_observation, info = env.reset()
    episode_reward = 0
    
    # Reset agent's temporal smoothing
    agent.reset()
    
    # Run episode
    for t in range(max_timesteps):
        # Get action from agent
        action = agent.sample_action(current_observation)
        
        # Take step in environment
        next_observation, reward, terminated, truncated, info = env.step(action)
        
        # Store transition and update episode reward
        agent.put_data(current_observation, action, reward, next_observation, 
                      terminated or truncated)
        episode_reward += reward
        
        # Update observation
        current_observation = next_observation
        
        # Check if episode is done
        if terminated or truncated:
            break
    
    # Train agent and update statistics
    policy_loss, value_loss = agent.train()
    agent.update_stats(episode_reward)
    
    # Print progress every 10 episodes
    if episode % 10 == 0:
        avg_reward = agent.get_average_reward()
        print(f"Episode {episode+1}, Reward: {episode_reward:.2f}, "
              f"Avg Reward: {avg_reward:.2f}, "
              f"Action STD: {agent.action_std:.3f}")
    
    # Track and plot
    tracker.track(info)
    if (episode + 1) % 10 == 0:
        tracker.plot(r_mean_=True, r_std_=True, r_sum=dict(linestyle=':', marker='x'))

env.close()

# Write log file
env.write_log(folder="logs", file="xxxx00-agent-log.txt")

A small demo with a predefined heuristic that is suboptimal and has no notion of balance (and is designed for the orignal BipedalWalker environment)...

In [ ]:
from gymnasium.envs.box2d.bipedal_walker import BipedalWalkerHeuristics

env = rld.make(
    "rldurham/Walker",
    # "BipedalWalker-v3",
    render_mode="human",
    # render_mode="rgb_array",
    hardcore=False,
    # hardcore=True,
)
_, obs, info = rld.seed_everything(42, env)

heuristics = BipedalWalkerHeuristics()

act = heuristics.step_heuristic(obs)
for _ in range(500):
    obs, rew, terminated, truncated, info = env.step(act)
    act = heuristics.step_heuristic(obs)
    if terminated or truncated:
        break
    if env.render_mode == "rgb_array":
        rld.render(env, clear=True)
env.close()